In [1]:
library(lme4)
library(simr)
library(dplyr)
library(broom)
install.packages("sjPlot")
library(sjPlot)
install.packages("broom.mixed")
library(broom.mixed)
library(htmlTable)
library(dplyr)
library(stringr)
library(car)
install.packages("stargazer")
library(stargazer)
install.packages("lmerTest")
library(lmerTest)

Loading required package: Matrix


Attaching package: ‘simr’


The following object is masked from ‘package:lme4’:

    getData



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpAMg2hs/downloaded_packages


Learn more about sjPlot with 'browseVignettes("sjPlot")'.




The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpAMg2hs/downloaded_packages



Attaching package: ‘stringr’


The following object is masked from ‘package:simr’:

    fixed


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpAMg2hs/downloaded_packages



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpAMg2hs/downloaded_packages



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




In [2]:


file_path_total <- 'pilot_total_ltm_stats/matching_new.csv'
# file_path_total <- 'pilot_total_ltm_stats/TEST_wm_ltm_p_pilot_total_with_predictions_and_rolling_matched.csv'
file_path_5 <- 'pilot5_ltm_stats/matching_new.csv'

file_path_6 <- 'pilot6_ltm_stats/matching_new.csv'


df_total <- read.csv(file_path_total) 
df5 <- read.csv(file_path_5) 
df6 <- read.csv(file_path_6) 


In [3]:
control <- lmerControl(optimizer = "bobyqa")

In [4]:
# run_ltm_models <- function(df) {
    
    
#     ltm_rel <- glmer(recall_answ ~   p_values_it_ltm + p_values_v2_ltm + it_pos_neg + v2_pos_neg + reliability_binary_z *  attend_binary_z  + tested_binary_z + (1 | participant), 
#                       data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
#                                             optCtrl = list(maxfun = 500000),
#                                             calc.derivs = FALSE))
#      return(list(
#     ltm_rel       = ltm_rel

#   ))
# }

In [5]:
run_ltm_models <- function(df) {
    


    ltm_rel <- glmer(recall_answ ~   p_values_it_flipped_ltm + p_values_v2_flipped_ltm + it_pos_neg_ltm + v2_pos_neg_ltm
                        + reliability_binary_z *  attend_binary_z  + tested_binary_z  + (1 | participant), 
                      data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))
     return(list(
    ltm_rel       = ltm_rel

  ))
}
results_total <- run_ltm_models(df_total)
results_old_stims <- run_ltm_models(df6)
results_new_stims <- run_ltm_models(df5)


In [6]:
# results_old_stims <- run_ltm_models(df6)


In [7]:
# results_new_stims <- run_ltm_models(df5)


In [8]:
library(htmlTable)

extract_significance <- function(model_summary) {
  coef_table <- model_summary$coefficients
  var_names <- rownames(coef_table)
  
  # Get p-values
  if ("Pr(>|z|)" %in% colnames(coef_table)) {
    p_values <- coef_table[, "Pr(>|z|)"]
  } else if ("Pr(>|t|)" %in% colnames(coef_table)) {
    p_values <- coef_table[, "Pr(>|t|)"]
  } else {
    stop("Cannot find p-values in coefficients table")
  }
  
  # Calculate significance stars
  significance <- ifelse(is.na(p_values), "",
                        ifelse(p_values < 0.001, "***",
                              ifelse(p_values < 0.01, "**",
                                    ifelse(p_values < 0.05, "*",
                                          ifelse(p_values < 0.1, ".", "")))))
  
  return(data.frame(
    variable = var_names,
    p_value = p_values,
    significance = significance,
    stringsAsFactors = FALSE
  ))
}

compare_models <- function(model_list, model_names, 
                          include_p_values = TRUE,
                          only_significant = TRUE) {
  
  # Extract significance for all models
  sig_list <- lapply(model_list, extract_significance)
  names(sig_list) <- model_names
  
  # Get all variables
  all_vars <- unique(unlist(lapply(sig_list, function(x) x$variable)))
  
  # Create result dataframe
  result_df <- data.frame(Variable = all_vars, stringsAsFactors = FALSE)
  
  # Add columns for each model
  for (model_name in model_names) {
    model_sig <- sig_list[[model_name]]
    
    result_df[[model_name]] <- sapply(all_vars, function(var) {
      idx <- which(model_sig$variable == var)
      if (length(idx) == 0) return("")
      
      p_val <- model_sig$p_value[idx]
      stars <- model_sig$significance[idx]
      
      if (is.na(p_val)) return("")
      
      if (include_p_values) {
        if (stars != "") {
          return(paste0(stars, "<br>", sprintf("%.4f", p_val)))
        } else {
          return(sprintf("%.4f", p_val))
        }
      } else {
        return(stars)
      }
    })
  }
  
  # Filter to only significant if requested
  if (only_significant) {
    # Keep rows where at least one model has significance
    has_sig <- apply(result_df[, -1, drop = FALSE], 1, function(row) {
      any(grepl("[*.]", row))
    })
    result_df <- result_df[has_sig, , drop = FALSE]
  }
  
  # Remove completely empty rows
  non_empty <- apply(result_df[, -1, drop = FALSE], 1, function(row) any(row != ""))
  result_df <- result_df[non_empty, , drop = FALSE]
  
  # Create HTML table
  html_table <- htmlTable(
    result_df,
    caption = "Model Comparison<br><small>*** p&lt;0.001, ** p&lt;0.01, * p&lt;0.05, . p&lt;0.1</small>",
    rnames = FALSE,
    css.cell = "padding: 6px; border: 1px solid #ddd;",
    css.table = "border-collapse: collapse; width: 100%; font-family: Arial, sans-serif;",
    css.header = "background-color: #f2f2f2; font-weight: bold;"
  )
  
  return(html_table)
}


In [9]:
ltm_rel_old <- summary(results_old_stims$ltm_rel)
ltm_rel_new <- summary(results_new_stims$ltm_rel)
ltm_rel_total <- summary(results_total$ltm_rel)

comparison_table_rel <- compare_models(
  model_list = list(ltm_rel_old, ltm_rel_new, ltm_rel_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,
  only_significant = TRUE
)

In [10]:
IRdisplay::display_html(comparison_table_rel)


Model Comparison *** p<0.001, ** p<0.01, * p<0.05, . p<0.1 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 0.2528 
 0.8784 
 0.5296 
 
 
 p_values_it_flipped_ltm 
 ** 0.0076 
 . 0.0528 
 ** 0.0011 
 
 
 p_values_v2_flipped_ltm 
 0.2293 
 0.4110 
 0.8207 
 
 
 it_pos_neg_ltm 
 * 0.0141 
 0.2295 
 ** 0.0061 
 
 
 v2_pos_neg_ltm 
 0.9292 
 0.1794 
 0.4134 
 
 
 reliability_binary_z 
 0.4821 
 0.1082 
 . 0.0945 
 
 
 attend_binary_z 
 0.1433 
 0.1032 
 * 0.0275 
 
 
 tested_binary_z 
 *** 0.0000 
 *** 0.0000 
 *** 0.0000 
 
 
 reliability_binary_z:attend_binary_z 
 0.3218 
 0.5830 
 0.7338

In [11]:
library(knitr)
library(kableExtra)
library(IRdisplay)

# Function to create combined results table for LTM models with different predictors
create_combined_results_table_ltm <- function(model_old, model_new, model_total, 
                                              model_name = "Combined Model Results",
                                              save_file = NULL) {
  
  # Function to clean predictor names
  clean_names <- function(names) {
    names <- gsub("p_values_v2_flipped_ltm", "V2_p_value", names)
    names <- gsub("p_values_it_flipped_ltm", "IT_p_value", names)
    names <- gsub("reliability_binary_z", "reliability", names)
    names <- gsub("attend_binary_z", "attention", names)
    names <- gsub("tested_binary_z", "tested", names)
    names <- gsub("it_pos_neg_ltm", "IT_sign", names)
    names <- gsub("v2_pos_neg_ltm", "V2_sign", names)
    return(names)
  }
  
  # Function to format p-value (vectorized, no leading zero)
  format_p <- function(p_vec) {
    sapply(p_vec, function(p) {
      if (p < 0.001) {
        return("&lt;.001")
      } else {
        formatted <- sprintf("%.3f", p)
        # Remove leading zero
        formatted <- sub("^0\\.", ".", formatted)
        return(formatted)
      }
    })
  }
  
  # Function to format to n significant figures
  format_sig_figs <- function(x, n = 2) {
    formatted <- signif(x, n)
    # Convert to character, preserving the significant figures
    as.character(formatted)
  }
  
  # Function to combine values with color formatting
  combine_colored <- function(old_val, new_val, total_val) {
    paste0(
      '<span style="color: purple;">', old_val, '</span>',
      ' / ',
      '<span style="color: blue;">', new_val, '</span>',
      ' / ',
      '<span style="color: black; font-weight: bold;">', total_val, '</span>'
    )
  }
  
  # Get coefficients for all three models
  coef_old <- coef(summary(model_old))
  coef_new <- coef(summary(model_new))
  coef_total <- coef(summary(model_total))
  
  # Get confidence intervals
  ci_old <- confint(model_old, method = "Wald", quiet = TRUE)
  ci_new <- confint(model_new, method = "Wald", quiet = TRUE)
  ci_total <- confint(model_total, method = "Wald", quiet = TRUE)
  
  # Get predictor names
  predictors <- clean_names(rownames(coef_total))
  
  # Check which predictors are significant in all three models (p < .05)
  sig_in_all <- (coef_old[, "Pr(>|z|)"] < 0.05) & 
                (coef_new[, "Pr(>|z|)"] < 0.05) & 
                (coef_total[, "Pr(>|z|)"] < 0.05)
  
  # Bold predictor names that are significant in all three
  predictors_formatted <- ifelse(sig_in_all, 
                                 paste0("<strong>", predictors, "</strong>"), 
                                 predictors)
  
  # Combine values with color formatting
  results <- data.frame(
    Predictor = predictors_formatted,
    Beta = combine_colored(
      format_sig_figs(coef_old[, "Estimate"], 2),
      format_sig_figs(coef_new[, "Estimate"], 2),
      format_sig_figs(coef_total[, "Estimate"], 2)
    ),
    SE = combine_colored(
      format_sig_figs(coef_old[, "Std. Error"], 2),
      format_sig_figs(coef_new[, "Std. Error"], 2),
      format_sig_figs(coef_total[, "Std. Error"], 2)
    ),
    z = combine_colored(
      format_sig_figs(coef_old[, "z value"], 2),
      format_sig_figs(coef_new[, "z value"], 2),
      format_sig_figs(coef_total[, "z value"], 2)
    ),
    p = combine_colored(
      format_p(coef_old[, "Pr(>|z|)"]),
      format_p(coef_new[, "Pr(>|z|)"]),
      format_p(coef_total[, "Pr(>|z|)"])
    ),
    CI_Lower = combine_colored(
      format_sig_figs(ci_old[rownames(coef_old), 1], 2),
      format_sig_figs(ci_new[rownames(coef_new), 1], 2),
      format_sig_figs(ci_total[rownames(coef_total), 1], 2)
    ),
    CI_Upper = combine_colored(
      format_sig_figs(ci_old[rownames(coef_old), 2], 2),
      format_sig_figs(ci_new[rownames(coef_new), 2], 2),
      format_sig_figs(ci_total[rownames(coef_total), 2], 2)
    ),
    stringsAsFactors = FALSE
  )
  
  # Create formatted table with borders
  table <- kable(results, row.names = FALSE, align = c("l", "r", "r", "r", "r", "r", "r"),
                 escape = FALSE, format = "html") %>%
    kable_styling(bootstrap_options = c("striped", "hover", "condensed", "bordered"),
                  full_width = FALSE) %>%
    add_header_above(c(" " = 1, 
                       "Study 1 / Study 2 / Combined" = 6))
  
  # Create header
  header_html <- paste0(
    '<h3>', model_name, '</h3>',
    '<p style="font-size: 12px;">',
    '<span style="color: purple;">■</span> Study 1 &nbsp;&nbsp; ',
    '<span style="color: blue;">■</span> Study 2 &nbsp;&nbsp; ',
    '<span style="color: black; font-weight: bold;">■</span> Combined',
    '<br><strong>Bold predictors</strong> are significant (p &lt; .05) in all three studies.',
    '</p>'
  )
  
  # Add custom CSS for clear borders
  css_style <- '
  <style>
    table {
      border-collapse: collapse !important;
      border: 2px solid #333 !important;
    }
    th, td {
      border: 1px solid #666 !important;
      padding: 8px !important;
    }
    th {
      background-color: #f0f0f0 !important;
      font-weight: bold !important;
    }
  </style>
  '
  
  # Combine HTML
  full_html <- paste0(css_style, header_html, as.character(table))
  
  # Display in notebook
  display_html(full_html)
  
  # Save to file if requested
  if (!is.null(save_file)) {
    complete_html <- paste0(
      '<!DOCTYPE html>
      <html>
      <head>
        <meta charset="utf-8">
        <title>', model_name, '</title>',
      css_style,
      '</head>
      <body>',
      full_html,
      '</body>
      </html>'
    )
    writeLines(complete_html, save_file)
    cat("\nTable saved to:", save_file, "\n")
  }
  
  return(results)
}




# Create the table using your existing model results
ltm_rel_combined <- create_combined_results_table_ltm(
  results_old_stims$ltm_rel,
  results_new_stims$ltm_rel,
  results_total$ltm_rel,
  "Model Results: LTM Recall",
  save_file = "ltm_rel_results.html"
)


Attaching package: ‘kableExtra’


The following object is masked from ‘package:dplyr’:

    group_rows


Warning message:
“'xfun::attr()' is deprecated.
Use 'xfun::attr2()' instead.
See help("Deprecated")”


Model Results: LTM Recall ■ Study 1    ■ Study 2    ■ Combined Bold predictors are significant (p < .05) in all three studies. 
 
 
 
 Study 1 / Study 2 / Combined 
 
 
 Predictor 
 Beta 
 SE 
 z 
 p 
 CI_Lower 
 CI_Upper 
 
 
 
 
 (Intercept) 
 1 / -0.14 / 0.4 
 0.89 / 0.93 / 0.64 
 1.1 / -0.15 / 0.63 
 .253 / .878 / .530 
 -0.73 / -2 / -0.86 
 2.8 / 1.7 / 1.7 
 
 
 IT_p_value 
 4.9 / 3.9 / 4.4 
 1.8 / 2 / 1.4 
 2.7 / 1.9 / 3.3 
 .008 / .053 / .001 
 1.3 / -0.047 / 1.8 
 8.5 / 7.9 / 7.1 
 
 
 V2_p_value 
 -3.7 / 2.5 / -0.49 
 3.1 / 3 / 2.1 
 -1.2 / 0.82 / -0.23 
 .229 / .411 / .821 
 -9.7 / -3.4 / -4.7 
 2.3 / 8.4 / 3.7 
 
 
 IT_sign 
 0.085 / 0.047 / 0.07 
 0.035 / 0.039 / 0.026 
 2.5 / 1.2 / 2.7 
 .014 / .230 / .006 
 0.017 / -0.03 / 0.02 
 0.15 / 0.12 / 0.12 
 
 
 V2_sign 
 0.0026 / 0.045 / 0.018 
 0.029 / 0.033 / 0.022 
 0.089 / 1.3 / 0.82 
 .929 / .179 / .413 
 -0.055 / -0.021 / -0.025 
 0.06 / 0.11 / 0.06 
 
 
 reliability 
 0.021 / 0.05 / 0.036 
 0.029 / 0.031 / 0.021 
 0.7 / 1.6 / 1.7 
 .482 / .108 / .095 
 -0.037 / -0.011 / -0.0062 
 0.078 / 0.11 / 0.078 
 
 
 attention 
 -0.052 / -0.06 / -0.056 
 0.035 / 0.037 / 0.026 
 -1.5 / -1.6 / -2.2 
 .143 / .103 / .028 
 -0.12 / -0.13 / -0.11 
 0.017 / 0.012 / -0.0062 
 
 
 tested 
 -0.58 / -0.59 / -0.59 
 0.037 / 0.039 / 0.027 
 -16 / -15 / -22 
 <.001 / <.001 / <.001 
 -0.65 / -0.67 / -0.64 
 -0.51 / -0.51 / -0.53 
 
 
 reliability:attention 
 -0.03 / 0.018 / -0.0074 
 0.03 / 0.032 / 0.022 
 -0.99 / 0.55 / -0.34 
 .322 / .583 / .734 
 -0.089 / -0.045 / -0.05 
 0.029 / 0.08 / 0.035


Table saved to: ltm_rel_results.html 


In [12]:
ltm_rel_new

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: recall_answ ~ p_values_it_flipped_ltm + p_values_v2_flipped_ltm +  
    it_pos_neg_ltm + v2_pos_neg_ltm + reliability_binary_z *  
    attend_binary_z + tested_binary_z + (1 | participant)
   Data: df
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 5e+05),  
    calc.derivs = FALSE)

     AIC      BIC   logLik deviance df.resid 
  6895.0   6964.0  -3437.5   6875.0     7323 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.5997  0.2218  0.3510  0.5483  1.8795 

Random effects:
 Groups      Name        Variance Std.Dev.
 participant (Intercept) 0.5546   0.7447  
Number of obs: 7333, groups:  participant, 134

Fixed effects:
                                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)                          -0.14202    0.92811  -0.153   0.8784    
p_values_it_flipped_ltm               3.90877  

In [13]:
# ltm_rel <- glmer(recall_answ ~   p_values_v2_flipped_ltm + p_values_it_flipped_ltm +
#                         + reliability_binary_z *  attend_binary_z  + tested_binary_z + it_pos_neg_ltm + v2_pos_neg_ltm + (1 | participant), 
#                       data = df_total, family = binomial,control = glmerControl(optimizer = "bobyqa",
#                                             optCtrl = list(maxfun = 500000),
#                                             calc.derivs = FALSE))


# summary(ltm_rel)

In [14]:

betas <- fixef(ltm_rel)

# If you want more detailed information including standard errors, t-values, etc.
summary_model <- summary(ltm_rel)
coef_table <- summary_model$coefficients
pred_response <- predict(ltm_rel, type = "response")


intercept_logit <- fixef(ltm_rel)[1]  # or coef(summary(ltm_rel))[1,1]

mean_ltm_accuracy <- plogis(intercept_logit)  # or exp(intercept_logit)/(1+exp(intercept_logit))
print(intercept_logit)
print(paste("Mean LTM Accuracy:", round(mean_ltm_accuracy, 4)))

# Convert to data frame for easier handling
betas_df <- data.frame(
  term = names(betas),
  beta = betas,
  row.names = NULL
)

# If you want the full coefficient table with SE, z-values, p-values:
full_coef_df <- data.frame(
  term = rownames(coef_table),
  beta = coef_table[, "Estimate"],
  se = coef_table[, "Std. Error"],
  z_value = coef_table[, "z value"],
  p_value = coef_table[, "Pr(>|z|)"],
  intercept = mean_ltm_accuracy,
  pred_response = pred_response[,"x"],
  row.names = NULL
)

# Save to CSV
write.csv(betas_df, "model_betas.csv", row.names = FALSE)
# Or for the full table:
write.csv(full_coef_df, "model_coefficients_full.csv", row.names = FALSE)

# To specifically get the beta for p_values_it_flipped_ltm:
beta_it_flipped <- betas["p_values_it_flipped_ltm"]
print(paste("Beta for p_values_it_flipped_ltm:", beta_it_flipped))

ERROR: Error in eval(expr, envir, enclos): object 'ltm_rel' not found


In [ ]:
pred_response <- predict(ltm_rel, type = "response")
print(pred_response)

In [ ]:
ltm_rel <- glmer(recall_answ ~   p_values_it_flipped_ltm + (1 | participant), 
                      data = df_total, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))

In [ ]:
ltm_rel <- glmer(recall_answ ~   p_values_it_flipped_ltm + (1 | participant), 
                      data = df_total, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))

newdat <- data.frame(p_values_it_flipped_ltm = seq(-1.2, 1.2, length.out = 1000000))
preds <- predict(ltm_rel, newdata = newdat, type = "response", re.form = NA)
write.csv(preds, "preds_response_it.csv", row.names = FALSE)





In [ ]:
df_total$rt_ltm <- df_total$image_recall_response.rt
df_correct <- subset(df_total, recall_answ == 1)
df_correct$log_rt_ltm <- log(df_correct$rt_ltm)

cols_to_scale <- c("reliability_binary_z", "attend_binary_z", "tested_binary_z",
                   "it_ltm_z", "it_ltm_sq_z", "it_ltm_cube_z", "v2_ltm_z", "v2_ltm_sq_z","v2_ltm_cube_z", 
              "it_ltm_diff_z", "it_ltm_diff_sq_z","it_ltm_diff_cube_z",  "v2_ltm_diff_z", "v2_ltm_diff_sq_z","v2_ltm_diff_cube_z", 
                  "it_ltm_diff_sign_z", "it_ltm_diff_sign_sq_z","it_ltm_diff_sign_cube_z", "v2_ltm_diff_sign_z", "v2_ltm_diff_sign_sq_z", "v2_ltm_diff_sign_cube_z")

df_correct[paste0(cols_to_scale, "_new")] <- lapply(df_correct[cols_to_scale], scale)
    
    
model_rel <- lmer(log_rt_ltm ~ p_values_v2_flipped_ltm + p_values_it_flipped_ltm +
                        + reliability_binary_z_new *  attend_binary_z_new  + tested_binary_z_new + (1 | participant), 
                      data = df_correct, control = control)

summary(model_rel)

